# 실습 내용

- 다양한 알고리즘으로 모델을 만들고 성능을 예측합니다.
- 성능이 좋을 것으로 판단된 모델의 성능을 튜닝합니다.
- 튜닝된 모델의 성능을 평가합니다.

# 1.환경 준비

- 기본 라이브러리와 대상 데이터를 가져와 이후 과정을 준비합니다.

In [1]:

# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings(action='ignore')
%config InlineBackend.figure_format = 'retina'


- 데이터 경로: https://raw.githubusercontent.com/jangrae/csv/master/Attrition_simple2.csv

In [2]:
# 데이터 읽어오기
path = 'https://raw.githubusercontent.com/jangrae/csv/master/Attrition_simple2.csv'
data = pd.read_csv(path)

# 2.데이터 이해

- 분석할 데이터를 충분히 이해할 수 있도록 다양한 탐색 과정을 수행합니다.

**데이터 설명**

- Attrition: 이직 여부 (1: 이직, 0: 잔류)
- Age: 나이
- DistanceFromHome: 집-직장 거리 (단위: 마일)
- EmployeeNumber: 사번	
- Gender: 성별 (Male, Female)
- JobSatisfaction: 직무 만족도(1: Low, 2: Medium, 3: High, 4: Very High)
- MaritalStatus: 결혼 상태 (Single, Married, Divorced)
- MonthlyIncome: 월급 (단위: 달러)
- OverTime: 야근 여부 (Yes, No)
- PercentSalaryHike: 전년 대비 급여 인상율(단위: %)
- TotalWorkingYears: 총 경력 연수

# 3.데이터 준비

- 전처리 과정을 통해 머신러닝 알고리즘에 사용할 수 있는 형태의 데이터를 준비합니다.
- 다음과 같은 전처리를 적절한 순서에 따라 진행합니다.
    - 불필요한 변수 제거
    - 필요한 변수 추가
    - 결측치 제거
    - x, y 분리
    - 가변수화
    - 학습용, 평가용 데이터 분리
    - 스케일링(정규화) 등

In [3]:

# 제거 대상: EmployeeNumber
drop_cols  = ['EmployeeNumber']

# 변수 제거
data.drop(drop_cols, axis=1, inplace=True)

# 확인
data.head()

# 확인


,Attrition,Age,DistanceFromHome,Gender,JobSatisfaction,MaritalStatus,MonthlyIncome,OverTime,PercentSalaryHike,TotalWorkingYears
0,0,33,7,Male,3,Married,11691,No,11,14
1,0,35,18,Male,4,Single,9362,No,11,10
2,0,42,6,Male,1,Married,13348,No,13,18
3,0,46,2,Female,1,Married,17048,No,23,28
4,1,22,4,Male,3,Single,3894,No,16,4


In [21]:
data

,Attrition,Age,DistanceFromHome,Gender,JobSatisfaction,MaritalStatus,MonthlyIncome,OverTime,PercentSalaryHike,TotalWorkingYears
0,0,33,7,Male,3,Married,11691,No,11,14
1,0,35,18,Male,4,Single,9362,No,11,10
2,0,42,6,Male,1,Married,13348,No,13,18
3,0,46,2,Female,1,Married,17048,No,23,28
4,1,22,4,Male,3,Single,3894,No,16,4
...,...,...,...,...,...,...,...,...,...,...
1191,0,32,5,Female,2,Married,5878,No,12,12
1192,0,27,19,Male,1,Divorced,4066,No,11,7
1193,0,29,9,Male,3,Married,2451,No,18,5
1194,0,29,2,Male,3,Married,4649,No,14,4


In [22]:

# target 확인

target = 'Attrition'

# 데이터 분리
x = data.drop(target, axis=1)
y = data.loc[:, target]




In [23]:
# 가변수화 대상: Gender, JobSatisfaction, MaritalStatus, OverTime

dumm_cols = ['Gender', 'JobSatisfaction', 'MaritalStatus', 'OverTime']
# 가변수화
x = pd.get_dummies(x ,columns=dumm_cols, drop_first=True)

# 확인




In [24]:
# 모듈 불러오기
from sklearn.model_selection import train_test_split

# 데이터 분리
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1)  



In [25]:
# 모듈 불러오기
from sklearn.preprocessing import MinMaxScaler

# 정규화
scaler = MinMaxScaler()
scaler.fit(x_train)
x_train_s = scaler.transform(x_train)
x_test_s = scaler.transform(x_test)

# 4.성능 예측

- 여러 알고리즘으로 모델을 만들고 K-Fold CV로 성능을 예측합니다.
- 다음과 같은 알고리즘 중에서 일부를 대상으로 합니다.

    - KNN
    
    - Random Forest
    - XGBoost
    - LigntGBM 등
- 각 모델의 예측된 성능 정보를 수집해 마지막에 비교합니다.
- 우선 이후 사용할 함수를 모두 불러옵니다.

In [26]:
# 불러오기
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import *

In [27]:

# 불러오기
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

# 선언하기 
model = KNeighborsClassifier()

# 검증하기
cv_score = cross_val_score(model, x_train_s, y_train,cv = 5, scoring = 'accuracy')

# 확인
print(cv_score)
print('평균:', cv_score.mean())

#성능정보 수집
result = {}
result['KNN'] = cv_score.mean()
print(result)


[0.875      0.8452381  0.81437126 0.8502994  0.83233533]
평균: 0.8434488166524094
{'KNN': 0.8434488166524094}


In [28]:

# 선언하기
model = RandomForestClassifier(max_depth=5, random_state =1)

cv_score = cross_val_score(model, x_train, y_train,cv = 5, scoring = 'accuracy')

# 확인
print(cv_score)
print('평균:', cv_score.mean())

#성능정보 수집

result['Random Forest'] = cv_score.mean()
print(result)



[0.83333333 0.85119048 0.83233533 0.85628743 0.86227545]
평균: 0.8470844026233246
{'KNN': 0.8434488166524094, 'Random Forest': 0.8470844026233246}


In [29]:

# 선언하기
model = XGBClassifier(max_depth = 5, random_state=1)

cv_score = cross_val_score(model, x_train, y_train,cv = 5, scoring = 'accuracy')

# 확인
print(cv_score)
print('평균:', cv_score.mean())


result['XGBoost'] = cv_score.mean()
print(result)

[0.83333333 0.83928571 0.82634731 0.85628743 0.82035928]
평균: 0.835122611919019
{'KNN': 0.8434488166524094, 'Random Forest': 0.8470844026233246, 'XGBoost': 0.835122611919019}


In [30]:

# 선언하기
model = LGBMClassifier(max_depth = 5, random_state=1)

cv_score = cross_val_score(model, x_train, y_train,cv = 5, scoring = 'accuracy')

# 확인
print(cv_score)
print('평균:', cv_score.mean())

result['LightGBM'] = accuracy_score(y_test, y_pred)
print(result)


[0.83333333 0.85714286 0.80838323 0.85628743 0.84431138]
평균: 0.8398916452808669
{'KNN': 0.8434488166524094, 'Random Forest': 0.8470844026233246, 'XGBoost': 0.835122611919019, 'LightGBM': 0.8272980501392758}


# 5.성능 비교

- 수집한 모델들의 성능 정보를 비교해  어떤 모델을 사용할 지 결정합니다.

In [ ]:
# 성능 시각화 비교
labels=df['algorithm']
values=df['scores'].round(2)
plt.title('Model Performance')
colors = sns.color_palette('Pastel2', len(labels))
bar=plt.bar(labels,values,color=colors)
plt.ylabel('Accuracy')
plt.xlabel('Model Name')
plt.ylim(0.2,0.91)
for idx,rect in enumerate(bar):
    plt.text(idx,rect.get_height()+0.01,values[idx],ha='center')
plt.xticks(rotation=45)
plt.show()



# 6.성능 튜닝

- 위에서 성능이 가장 좋을 것으로 예측된 모델을 튜닝합니다.
- 본 실습에서는 XGBoost 모델 성능을 튜닝합니다.

In [34]:
model_dt = RandomForestClassifier( random_state =1)

cv_score = cross_val_score(model_dt, x_train, y_train, cv =5 , scoring='accuracy')

print(cv_score)
print(cv_score.mean())

# 기본 모델 선언

# 파라미터 지정
  # max_depth: range(1, 21)
param={'max_depth':range(1,21)}

# Random Search 선언
  # cv=5
  # scoring='r2'


model=GridSearchCV(model_dt,param,
                   cv=5,
                   scoring='accuracy')

# 모델 선언



[0.8452381  0.85714286 0.82035928 0.85628743 0.8502994 ]
0.8458654120330766


# 7.성능 평가

- 최적 파라미터로 학습된 모델에 대해 예측과 평가를 진행합니다.

In [35]:
model.fit(x_train, y_train)

y_pred = model.predict(x_test)

# 중요 정보 확인
print('=' * 80)
print(model.cv_results_['mean_test_score'])
print('-' * 80)
print('최적파라미터:', model.best_params_)
print('-' * 80)
print('최고성능:', model.best_score_)
print('=' * 80)


[0.83751782 0.83991303 0.84468919 0.8470844  0.8470844  0.84709153
 0.85187482 0.85665098 0.84946536 0.85305104 0.84587967 0.84230824
 0.85184631 0.8506487  0.84826775 0.84467494 0.84348446 0.84586541
 0.84586541 0.84586541]
--------------------------------------------------------------------------------
최적파라미터: {'max_depth': 8}
--------------------------------------------------------------------------------
최고성능: 0.856650983746792


# 8. 옵션: 클래스 불균형 문제 해결

- Under Sampling 또는 Over Samplig 후 모델 성능이 좋아지는 지 궁금합니다.
- Under Sampling 또는 Over Samplig 후 학습, 예측, 평가를 진행합니다.

In [36]:
# 불러오기
from imblearn.under_sampling import RandomUnderSampler

# Under Sampling
under_sample = RandomUnderSampler()
u_x_train, u_y_train = under_sample.fit_resample(x_train, y_train)

# 확인
print('전:', np.bincount(y_train))
print('후:', np.bincount(u_y_train))
# 학습하기
model.fit(u_x_train, u_y_train)

# 예측하기
y_pred = model.predict(x_test)

# 평가하기
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

전: [701 136]
후: [136 136]
[[208  92]
 [ 17  42]]
              precision    recall  f1-score   support

           0       0.92      0.69      0.79       300
           1       0.31      0.71      0.44        59

    accuracy                           0.70       359
   macro avg       0.62      0.70      0.61       359
weighted avg       0.82      0.70      0.73       359

